In [ ]:
# Loading Data
import numpy as np
import math
import random
import tqdm
import copy
import torch
import torch.nn as nn
import gpytorch
import warnings
from matplotlib import pyplot as plt
import pandas as pd

# Make plots inline
%matplotlib inline
import urllib.request
import os
from scipy.io import loadmat
from math import floor
from utils.tools import StandardScaler

#test Razvan's model
from models.ours import PA

def setup_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
# set seed
setup_seed(42)

warnings.filterwarnings('ignore')

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

pre_horizon = 12
seq_len = pre_horizon
pred_len = 1
num_nodes = 9

data1 = pd.read_csv("IMV_LSTM/SML2010/NEW-DATA-1.T15.txt", sep=' ')
data2 = pd.read_csv("IMV_LSTM/SML2010/NEW-DATA-2.T15.txt", sep=' ')
target = '3:Temperature_Comedor_Sensor'

'''
cols = [
    '3:Temperature_Comedor_Sensor',
 '4:Temperature_Habitacion_Sensor',
 '5:Weather_Temperature',
 '6:CO2_Comedor_Sensor',
 '7:CO2_Habitacion_Sensor',
 '8:Humedad_Comedor_Sensor',
 '9:Humedad_Habitacion_Sensor',
 '10:Lighting_Comedor_Sensor',
 '11:Lighting_Habitacion_Sensor',
 '12:Precipitacion',
 '13:Meteo_Exterior_Crepusculo',
 '14:Meteo_Exterior_Viento',
 '15:Meteo_Exterior_Sol_Oest',
 '16:Meteo_Exterior_Sol_Est',
 '20:Exterior_Entalpic_2',
 '21:Exterior_Entalpic_turbo',
 '22:Temperature_Exterior_Sensor']
'''


cols = [
    '3:Temperature_Comedor_Sensor',
 '4:Temperature_Habitacion_Sensor',
 '5:Weather_Temperature',
 '6:CO2_Comedor_Sensor',
 '7:CO2_Habitacion_Sensor',
 '8:Humedad_Comedor_Sensor',
 '9:Humedad_Habitacion_Sensor',
 '10:Lighting_Comedor_Sensor',
 '11:Lighting_Habitacion_Sensor']


df_data = pd.concat([data1, data2])[cols[:num_nodes]]
train_split = 3200
test_split = train_split+537
train_data = df_data[:train_split]

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
data = (df_data - mean)/std
data = torch.Tensor(data.values)
print(data.shape)
#data = torch.from_numpy(data).unsqueeze(-1) ## unsqueeze(-1)
new_data = data.unfold(0, seq_len+pred_len, 1).transpose(1,2)
train = new_data[:train_split-pre_horizon][::12]
test = new_data[train_split-pre_horizon:test_split-pre_horizon]
val = new_data[test_split-pre_horizon:]

train_x = train[:,:seq_len,]
train_y = train[:,seq_len:,].squeeze(-2)
test_x = test[:,:seq_len]
test_y = test[:,seq_len:,].squeeze(-2)
val_x = val[:,:seq_len]
val_y = val[:,seq_len:,].squeeze(-2)

full_train_i = torch.full((train_x.shape[0],1), dtype=torch.long, fill_value=0) ##
for i in range(1,train_x.shape[-1]):
    train_i_task = torch.full((train_x.shape[0],1), dtype=torch.long, fill_value=i)##
    full_train_i = torch.cat([full_train_i, train_i_task], 1) ##
full_train_i = full_train_i.reshape(-1).cuda()

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
    test = test.cuda()
    val_x, val_y = val_x.cuda(), val_y.cuda()
    
feature_extractor = PA(horizon=pred_len, lag=seq_len, dynamic=False, supports=None,
                       patch_sizes=[1], channels=32, num_nodes=num_nodes, 
                       input_dim=1, output_dim=1, device='cuda:0')

train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape, full_train_i.shape

In [ ]:
##########################################################################################
        
base_kernel_set = [gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()),\
                   gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel()),\
                   gpytorch.kernels.ScaleKernel(gpytorch.kernels.LinearKernel()),\
                   gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel())]
R = 1

final_kernel = 0
for i in range(R):
    #this_iter = base_kernel_set.copy()
    add_kernel = copy.deepcopy(base_kernel_set[0])
    for j in range(1,len(base_kernel_set)):
        add_kernel += copy.deepcopy(base_kernel_set[j])
    add_kenrel = gpytorch.kernels.ScaleKernel(add_kernel)

    if i == 0:
        mul_kernel = copy.deepcopy(add_kernel)
        final_kernel = copy.deepcopy(mul_kernel)
        continue
    else:
        mul_kernel *= copy.deepcopy(add_kernel)
        final_kernel += copy.deepcopy(mul_kernel)


print(final_kernel)

'''
final_kernel =  gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()) + \
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) + \
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()*gpytorch.kernels.PeriodicKernel()) + \
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()*gpytorch.kernels.PeriodicKernel()) + \
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()*gpytorch.kernels.RBFKernel())
'''
##########################################################################################

In [3]:
setup_seed(42)

idx = []
# The forward method
from gpytorch.models import ApproximateGP

class MultitaskGPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            #self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-10., 10.)
            self.covar_module = final_kernel
            self.task_covar_module = gpytorch.kernels.IndexKernel(num_tasks=num_nodes, rank=1)
            self.feature_extractor = feature_extractor

        def forward(self, x, index): 
            projected_x, _ = self.feature_extractor(x)
            projected_x = projected_x.squeeze()
            projected_x = projected_x.reshape(-1)
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            covar_i = self.task_covar_module(index) ##
            covar = covar_x.mul(covar_i) ##
            return gpytorch.distributions.MultivariateNormal(mean_x, covar) ##
    
#likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_nodes) ##
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MultitaskGPModel((train_x, full_train_i), train_y.reshape(-1), likelihood) ##


if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

In [ ]:
# Training the model
training_iterations = 200

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
min_val_loss = 9999
flag = 0
retrain = 0
torch.save(model.state_dict(), "./saved/Trigp/ETTh1.pth")
original = model.covar_module

full_val_i = torch.full((val_x.shape[0],1), dtype=torch.long, fill_value=0) ##
for j in range(1,num_nodes): ##
    val_i_task = torch.full((val_x.shape[0],1), dtype=torch.long, fill_value=j)##
    full_val_i = torch.cat([full_val_i, val_i_task], 1) ##
full_val_i = full_val_i.reshape(-1).cuda()##
print(full_val_i.shape, val_x.shape)

#full_test_i = torch.full((test_x.shape[0],1), dtype=torch.long, fill_value=0) ##
#for j in range(1,num_nodes): ##
#    test_i_task = torch.full((test_x.shape[0],1), dtype=torch.long, fill_value=j)##
#    full_test_i = torch.cat([full_test_i, test_i_task], 1) ##
#full_test_i = full_test_i.reshape(-1).cuda()##
#print(full_test_i.shape, test_x.shape)
            
def train():
    patience = 100
    counter = 0
    val_losses = []
    
    iterator = tqdm.notebook.tqdm(range(training_iterations))
    for i in iterator:
        # Find optimal model hyperparameters
        model.train()
        likelihood.train()

        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        outputs = model(train_x,full_train_i)
        # Calc loss and backprop derivatives
        train_loss = -mll(outputs, train_y.reshape(-1))
        train_loss.backward()
        iterator.set_postfix(train_loss=train_loss.item())
        optimizer.step()
        
        model.eval()
        likelihood.eval()
        with torch.no_grad():
            preds = model(val_x, full_val_i)
            val_loss = -mll(preds, val_y.reshape(-1))
            #preds = model(test_x, full_test_i)
            #val_loss = -mll(preds, test_y.reshape(-1))
            
            val_losses.append(val_loss)
            
        global min_val_loss
        if min_val_loss > val_loss and val_loss > 0:
            min_val_loss = val_loss
            global flag
            global original
            original = model.covar_module
            flag = 1
            print(min_val_loss)
            if retrain == 1:
                print("Saving...")
                torch.save(model.state_dict(), "./saved/Trigp/ETTh1.pth")
            counter = 0
        else:
            counter += 1
            if counter % 5 == 0:
                for params in optimizer.param_groups:
                    params['lr'] *= 0.9
                    print(params['lr'])
                    
        if counter == patience:
            break


train()

In [ ]:
##########################################################################################
## Retrain:
retrain = 1
constrains = []
for constraint_name, constraint in model.named_constraints():
    constrains.append(constraint)


index = 0
d = len(base_kernel_set)
a1 = len(base_kernel_set)
an = a1+(R-1)*d
length = int((a1+an)*R/2)
kernel_weights = torch.zeros(length)
for param_name, param in model.named_parameters():
    if 'covar_module' and 'raw_outputscale' in param_name:
        #print(f'Parameter name: {param_name:42}')
        param_name = param_name.split('.')
        constrain = constrains[index]
        kernel_weights[index] = constraint.transform(param)
        index += 1
        #print(constraint.transform(param))

pre = 0
now = a1
numbers = 1
best_kernel_index = torch.zeros((R,numbers)).int()
for r in range(R):
    weights = kernel_weights[pre:pre+d]
    pre = now
    now = a1 + a1+(r+1)*d
    weights = nn.functional.softmax(weights, dim=0)
    _, kernel_index = torch.sort(weights, descending=True)
    best_kernel_index[r] = kernel_index[:numbers]

#print(best_kernel_index)
####################################################################################

final_kernel = 0
for i in range(R):
    add_kernel = copy.deepcopy(base_kernel_set[best_kernel_index[i][0]])
    for j in range(1, numbers):
        add_kernel += copy.deepcopy(base_kernel_set[best_kernel_index[i][j]])

    if i == 0:
        mul_kernel = copy.deepcopy(add_kernel)
        final_kernel = copy.deepcopy(mul_kernel)
        continue
    else:
        mul_kernel *= copy.deepcopy(add_kernel)
        final_kernel += copy.deepcopy(mul_kernel)


print(final_kernel)

min_val_loss = 9999
flag = 0
model.covar_module = final_kernel.cuda()
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
#torch.save(model.state_dict(), "./saved/Trigp/ETTh1.pth")
train()



##########################################################################################

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
setup_seed(42)

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
# Making Predictions
MAE = []
RMSE = []
MAPE = []
model.load_state_dict(torch.load("./saved/Trigp/ETTh1.pth"))
model.eval()
likelihood.eval()
predictions = 0
        
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    for i in range(pre_horizon):
        if i == 0:
            test_x = test[i:-pre_horizon+i,:seq_len].clone()
        test_y = test[i:-pre_horizon+i,seq_len].squeeze().clone()
        
        full_test_i = torch.full((test_x.shape[0],1), dtype=torch.long, fill_value=0) ##
        for j in range(1,num_nodes): ##
            test_i_task = torch.full((test_x.shape[0],1), dtype=torch.long, fill_value=j)##
            full_test_i = torch.cat([full_test_i, test_i_task], 1) ##
        full_test_i = full_test_i.reshape(-1).cuda()##
        preds = model(test_x,full_test_i)
        print(test_x.shape, preds.mean.reshape(-1,num_nodes).unsqueeze(1).shape)
        
        real_preds = preds.mean[0::num_nodes]
        real_test_y = test_y[:,0]
        
        #print(real_preds.shape)
        #print("real_test_y", real_test_y.shape)
        real_test_y = real_test_y.cpu()*std[0]+mean[0]
        real_preds = real_preds.cpu()*std[0]+mean[0]
            
        rmse = mean_squared_error(real_test_y.cpu(), real_preds.cpu())
        mae = mean_absolute_error(real_test_y.cpu(), real_preds.cpu())
        mape = mean_absolute_percentage_error(real_test_y.cpu(), real_preds.cpu())
        MAE.append(mae)
        RMSE.append(rmse)
        MAPE.append(mape)
        test_x = torch.cat([test_x.squeeze().clone(), preds.mean.reshape(-1,num_nodes).unsqueeze(1)], axis=1)
        test_x = test_x[:,1:,:].clone()
        
        
MAE = np.array(MAE)
RMSE = np.array(RMSE)**0.5
MAPE = np.array(MAPE)

print(MAE.mean(), RMSE.mean(), MAPE.mean())